## Lab05
### Use Lesk algorighmn to classify words into right category

Lesk Algorithm – Disambiguating EVP entries to WordNet <br>
Recall that LA relies on the number of shared words to determine the intended sense of a
given context. It is not uncommon to have a tie with two or more senses having the same
number of shared words with the context. In order to break the tie, people have used
the weight such as idf (inverse document frequency) of word to break a tie and even to
improve the performance. Therefore, in the training phrase (preprocessing), we read the
dataset file and compute the following information:
- Document frequency (df): the number of word sense categories where a certain
word appears in. For instance, the word money appears in 80 categories including
get.v.01. Let D be the total number of categories. Then id f = d f /D.
- Within document term frequency: For each defining/example word, compute the
WordNet sense categories the word appears in.


- TF: 指某個字詞在該檔案中出現的次數/該檔案所有的字數量
- IDF: df取倒數後對其做log10
- tf-idf: tf * idf
- Step 1 : 計算senseDef裡的TF-IDF
- tf ：在字詞在各類 wncat 出現的次數/各類 wncat所有的字詞數量
- df ：在字詞出現在幾個 wncat 中/ wncat類別總數
- idf：df 倒數取log10
- Step 2: split data into 9:1 and train model

In [2]:
#from nltk.corpus import wordnet as wn
import re
from collections import defaultdict, Counter
from nltk.stem.wordnet import WordNetLemmatizer
lmtzr = WordNetLemmatizer()
import nltk, random

# 所有feature都分割成一個字
def words(text): return re.findall(r'\w+', text.lower())
    
TF = defaultdict(lambda: defaultdict(lambda: 0))
DF = defaultdict(lambda: [])
wncat_count = defaultdict(lambda: 0)

#詞性簡寫
def wnTag(pos): return {'noun': 'n', 'verb': 'v', 'adjective': 'a', 'adverb': 'r'}[pos]

def isHead(head, word, tag):
    try:
        return lmtzr.lemmatize(word, tag) == head  #詞型還原
    except:
        return False
training = [line.strip().split('\t') for line in open(r'C:/Users/asus/Downloads/nlp/Lab5/wn.in.evp.cat.txt', 'r', encoding = 'utf8') if line.strip() != '' ]
for wnid, wncat, senseDef, target in training:
    head, pos = wnid.split('-')[:2] #get單字 詞性
    for word in words(senseDef): #把所有單字切割
        if word != head and not isHead(head, word, pos): #單字或單字原型不存在時
            TF[word][wncat] += 1 #[單字原型][類別answer]+1     #該字詞在每一類出現次數
            DF[word] += [] if wncat in DF[word] else [wncat] #該字詞出現在那些類別
            wncat_count[wncat]+=1

In [3]:
##TFIDF 
# tf ：字詞在各類 wncat 出現的次數/各類 wncat所有的字詞數量
# df ：字詞出現在幾個 wncat中/ wncat類別總數  
# idf：df 倒數取log10
import math
def tfidf(word, wncat):
    tf = TF[word][wncat]/wncat_count[wncat]
    df = (len(DF[word]))+1/len(wncat_count)
    idf = math.log10(1/df)
    return tf*idf

## testing
print("TFIDF of (parking, get_rid_of.v.01) is", tfidf("parking", "get_rid_of.v.01"))
print("TFIDF of (lot, get_rid_of.v.01) is", tfidf("parking", "get_rid_of.v.01"))  

##取得每個字的TF-IDF
#將training每一行的第二個(senseDef)切割, 計算每個字的TFIDF
#{'cucumber': -0.002007382871432638, 'shaped': -0.010034365456100688...}
def feature_format(line):
    feature_dict={}
    for word in words(line[2]): 
        feature_dict.update({word: tfidf(word, line[1])}) #format to dictionary
    return (feature_dict, line[1])  #多個單字會對應到一個category

## Get feature
feature = [feature_format(line) for line in training]
print("feature[0]: ", feature[0])  #for test

TFIDF of (parking, get_rid_of.v.01) is -0.010101166546613746
TFIDF of (lot, get_rid_of.v.01) is -0.010101166546613746
feature[0]:  ({'lot': -0.016182463266751875, 'behind': -0.016071399781723568, 'in': -0.09506174447760563, 'the': -0.230895736715218, 'abandoned': -0.0, 'parking': -0.010101166546613746, 'leave': -0.015720270738149425, 'car': -0.03928251378163691, 'forsake': -0.004819928076979694, 'we': -0.04687165711478301, 'old': -0.11958360613925677, 'abandon': -0.0, 'empty': -0.011577162612865805}, 'get_rid_of.v.01')


In [4]:
import nltk, random
from nltk.probability import DictionaryProbDist as D  
from nltk.classify import SklearnClassifier 
from sklearn.linear_model import LogisticRegression

# split the feature set into 9:1
split_ratio = int(len(feature)*9/10)
train, test = feature[:split_ratio], feature[split_ratio:]
# train with SKLearn
sklearn_classifier = SklearnClassifier(LogisticRegression(C=10e5)).train(train)

In [21]:
print("Sklearn accuracy: %s" % (nltk.classify.accuracy(sklearn_classifier, test)))

Sklearn accuracy: 0.5344827586206896


In [62]:
## compare the answer and the predict result
# 剩下的10% test計算correct的次數
# accuracy= correct/number of test
correct=0
c=[]; f=[]
for test_feature, result in test:
    #用 test_feature預測結果 看跟result是否相同
    chk_result = sklearn_classifier.prob_classify(test_feature)._prob_dict
    # 機率最高的為結果
    predict = sorted(chk_result.items(), key= lambda x: x[1], reverse=True)[0][0]
    c.append(result)
    f.append(test_feature)
    if result == predict:
        correct= correct+1  # count correct
#     else print(result)
print(c)
print("Accuracy:")
print(correct/ len(test))

Accuracy:
0.0


In [83]:
def testLesk():
    testdata = [  line.strip().split('\t') for line in open(r'C:/Users/asus/Downloads/nlp/Lab5/wn.in.evp.cat.txt', 'r', encoding = 'utf8') if line.strip() != '' ]
    
    def df(word): return len(DF[word])
    def leskOverlap(senseDef, target):
        wnidCount = [ (wncat, tf, word, len(DF[word])+1) for word in senseDef \
                                                    for wncat, tf in TF[word].items() \
                                                    if wncat in target.values() ]
        res = sorted( [ (wnid, tf*int(1000/df)) for wnid, tf, word, df in wnidCount], key = lambda x: -x[1])[:7]
        if not res: return 'Not found'
        counter = Counter()
        for wnid, tfidf in res: counter[wnid] += tfidf
        return counter.most_common(1)[0]
        
    for evpid, wncat, senseDef, target in testdata:
        #head, pos = evpid.split('-')[:2]
        print( '%s\t%s\t%s' %(evpid, leskOverlap(words(senseDef), eval(target)), senseDef.split('||')[0] ) )

In [84]:
testLesk()

abandon-v-1	('get_rid_of.v.01', 466)	abandon
abandon-v-2	('abandon.v.02', 1725)	abandon give_up
abandon-v-3	('leave.v.01', 1507)	vacate empty abandon
abandon-v-4	('abandon.v.04', 1110)	abandon give_up
abandon-v-5	('leave.v.02', 1083)	abandon forsake desolate desert
abandoned-a-1	('uninhabited.a.01', 4199)	abandoned derelict deserted
abandoned-a-2	('uninhibited.a.01', 1311)	abandoned
ability-n-1	('quality.n.01', 2792)	ability
ability-n-2	('cognition.n.01', 3142)	ability power
abnormal-a-1	('abnormal.a.01', 2900)	abnormal unnatural
abnormal-a-2	('abnormal.a.01', 1620)	abnormal
abnormal-a-3	('abnormal.a.01', 1695)	abnormal
aboard-r-1	('adv.all', 1690)	aboard on_board
aboard-r-2	('adv.all', 3019)	aboard on_base
aboard-r-3	('adv.all', 3013)	aboard alongside
aboard-r-4	('adv.all', 2072)	aboard
abolish-v-1	('abolish.v.01', 1230)	abolish get_rid_of
abortion-n-1	('change.n.03', 1937)	abortion
abortion-n-2	('happening.n.01', 881)	miscarriage abortion
about-r-1	('adv.all', 15000)	approximately ab

act-v-7	('act.v.01', 768)	work act
act-v-8	('act.v.01', 584)	act
act-v-9	('dissemble.v.03', 2332)	dissemble pretend act
act-v-10	('make.v.03', 5250)	act play roleplay playact
act-n-1	('legal_document.n.01', 1977)	act enactment
act-n-2	('act.n.02', 3344)	act deed human_action human_activity
act-n-3	('act.n.02', 2007)	act
act-n-4	('act.n.02', 3936)	act routine number turn bit
act-n-5	('act.n.02', 4104)	act
action-n-1	('activity.n.01', 1259)	action
action-n-2	('state.n.02', 3464)	action activity activeness
action-n-3	('group_action.n.01', 1424)	military_action action
action-n-4	('natural_process.n.01', 2820)	natural_process natural_action action activity
action-n-5	('state.n.02', 548)	action
action-n-6	('trait.n.01', 6580)	action
action-n-7	('device.n.01', 2146)	action action_mechanism
action-n-8	('group_action.n.01', 2410)	legal_action action action_at_law
action-n-9	('group_action.n.01', 2660)	action
action-n-10	('activity.n.01', 2687)	action
active-a-1	('active.a.01', 573)	active
activ

alike-r-1	('adv.all', 4131)	alike likewise
alike-r-2	('adv.all', 1336)	alike
alive-a-1	('aware.a.01', 1660)	alive live
alive-a-2	('aware.a.01', 996)	alive
alive-a-3	('animated.a.01', 5612)	animated alive
alive-a-4	('aware.a.01', 664)	alive
alive-a-5	('existent.a.01', 1298)	active alive
alive-a-6	('aware.a.01', 5418)	alert alive awake
alive-a-7	('active.a.12', 1500)	alive live
all-r-1	('adv.all', 6325)	wholly entirely completely totally all altogether whole
allegation-n-1	('statement.n.01', 2148)	allegation
allegation-n-2	('statement.n.01', 1372)	allegation allegement
allege-v-1	('insist.v.01', 3724)	allege aver say
alleged-a-1	('declared.a.01', 692)	alleged
alleged-a-2	('questionable.a.01', 2002)	alleged so-called supposed
allegedly-r-1	('declared.a.01', 585)	allegedly
alliance-n-1	('state.n.02', 2714)	alliance confederation
alliance-n-2	('relation.n.01', 3464)	alliance bond
alliance-n-3	('organization.n.01', 1946)	alliance coalition alignment alinement
alliance-n-4	('relation.n.01', 1

architecture-n-3	('activity.n.01', 1941)	architecture
architecture-n-4	('property.n.02', 1840)	computer_architecture architecture
area-n-1	('area.n.01', 2965)	area country
area-n-2	('state.n.02', 446)	area
area-n-3	('body_part.n.01', 1986)	area region
area-n-4	('state.n.02', 6104)	sphere domain area orbit field arena
area-n-5	('structure.n.01', 1270)	area
area-n-6	('property.n.02', 1430)	area expanse surface_area
argue-v-1	('express.v.02', 553)	argue reason
argue-v-2	('communicate.v.02', 804)	argue contend debate fence
argue-v-3	('express.v.02', 882)	argue indicate
argument-n-1	('statement.n.01', 2605)	argument statement
argument-n-2	('act.n.02', 9245)	controversy contention contestation disputation disceptation tilt argument arguing
argument-n-3	('speech.n.02', 1268)	argument argumentation debate
argument-n-4	('statement.n.01', 1861)	argument literary_argument
argument-n-5	('code.n.03', 1119)	argument parameter
argument-n-6	('idea.n.01', 1490)	argument
arid-a-1	('dry.a.01', 4908)	arid

balance-v-3	('hold.v.14', 365)	poise balance
balance-v-4	('be.v.01', 348)	balance
balcony-n-1	('structure.n.01', 1324)	balcony
balcony-n-2	('structure.n.01', 1807)	balcony
bald-a-1	('overt.a.01', 2231)	bald barefaced
bald-a-2	('hairless.a.01', 1780)	bald bald-headed bald-pated
bald-a-3	('bare.a.04', 1378)	bald denuded denudate
ball-n-1	('equipment.n.01', 1426)	ball
ball-n-2	('weapon.n.01', 2500)	musket_ball ball
ball-n-3	('attribute.n.02', 1814)	ball globe orb
ball-n-4	('gathering.n.01', 1608)	ball
ball-n-5	('body_part.n.01', 6666)	testis testicle orchis ball ballock bollock nut egg
ball-n-6	('artifact.n.01', 2575)	ball
ball-n-7	('actor.n.01', 1533)	Ball Lucille_Ball
ball-n-8	('collection.n.01', 4366)	ball clod glob lump clump chunk
ballet-n-1	('dancing.n.01', 838)	ballet concert_dance
ballet-n-2	('music.n.01', 331)	ballet
balloon-n-1	('aircraft.n.01', 1391)	balloon
balloon-n-2	('artifact.n.01', 2279)	balloon
ban-v-1	('request.v.01', 1614)	ban censor
ban-v-2	('request.v.01', 861)	ban
b

...

struggle-v-3	('move.v.03', 3333)	clamber scramble shin shinny skin struggle sputter
struggle-v-4	('contend.v.06', 4215)	contend fight struggle
struggle-n-1	('group_action.n.01', 1336)	struggle battle
struggle-n-2	('group_action.n.01', 7254)	conflict struggle battle
struggle-n-3	('activity.n.01', 1570)	struggle
stubborn-a-1	('stubborn.a.01', 2650)	stubborn obstinate unregenerate
stubborn-a-2	('intractable.a.01', 3166)	refractory stubborn
stubbornness-n-1	('trait.n.01', 9122)	stubbornness obstinacy obstinance mulishness
stubbornness-n-2	('trait.n.01', 4020)	stubbornness bullheadedness obstinacy obstinance pigheadedness self-will
stuck-a-1	('stuck.a.01', 149)	stuck
stuck-a-2	('perplexed.a.01', 1556)	stuck
student-n-1	('person.n.01', 6668)	student pupil educatee
student-n-2	('person.n.01', 4821)	scholar scholarly_person bookman student
studio-n-1	('workplace.n.01', 5021)	studio
studio-n-2	('structure.n.01', 1083)	studio_apartment studio
studio-n-3	('workplace.n.01', 4666)	studio
stuff-n-1	

take-v-28	('put.v.01', 6960)	aim take train take_aim direct
take-v-29	('take.v.21', 710)	take
take-v-30	('have.v.02', 998)	carry pack take
take-v-31	('get.v.01', 1422)	lease rent hire charter engage take
take-v-32	('get.v.01', 1859)	subscribe subscribe_to take
take-v-33	('think.v.03', 680)	take
take-v-34	('change.v.02', 535)	take
take-v-35	('connect.v.01', 918)	take have
take-v-36	('act.v.01', 254)	claim take
take-v-37	('be.v.01', 1034)	accept take
take-v-38	('be.v.01', 2034)	contain take hold
take-v-39	('change.v.02', 1027)	take
take-v-40	('travel.v.01', 2251)	drive take
take-v-41	('think.v.03', 844)	take
take-v-42	('change.v.02', 3410)	contract take get
take-n-1	('income.n.01', 4094)	return issue take takings proceeds yield payoff
take-n-2	('activity.n.01', 3004)	take
takeout-n-1	('food.n.02', 2900)	takeout takeout_food takeaway
takeout-n-2	('statement.n.01', 1475)	takeout
takeover-n-1	('group_action.n.01', 3982)	coup_d'etat coup putsch takeover
takeover-n-2	('act.n.02', 488)	takeove

track-n-10	('object.n.01', 1032)	track cart_track cartroad
track-n-11	('object.n.01', 744)	track running
track-v-1	('travel.v.01', 894)	track
track-v-2	('travel.v.01', 762)	track
track-v-3	('travel.v.01', 8412)	chase chase_after trail tail tag give_chase dog go_after track
track-v-4	('travel.v.01', 8706)	traverse track cover cross pass_over get_over get_across cut_through cut_across
track-v-5	('travel.v.01', 578)	track
tractor-n-1	('wheeled_vehicle.n.01', 7569)	tractor
tractor-n-2	('motor_vehicle.n.01', 1888)	tractor
trade-n-1	('commerce.n.01', 2893)	trade
trade-n-2	('activity.n.01', 3569)	trade craft
trade-n-3	('activity.n.01', 3336)	trade patronage
trade-n-4	('group_action.n.01', 1380)	deal trade business_deal
trade-n-5	('activity.n.01', 2200)	craft trade
trade-n-6	('wind.n.01', 1008)	trade_wind trade
trade-n-7	('commerce.n.01', 2768)	barter swap swop trade
trade-v-1	('trade.v.01', 585)	trade merchandise
trade-v-2	('transfer.v.05', 977)	trade trade_in
trade-v-3	('be.v.01', 872)	trade

undoubtedly-r-1	('adv.all', 2362)	undoubtedly doubtless doubtlessly
undress-v-1	('remove.v.01', 7000)	undress discase uncase unclothe strip strip_down disrobe peel
undress-v-2	('remove.v.01', 5196)	strip undress divest disinvest
undressed-a-1	('unfinished.a.01', 1921)	undressed
undressed-a-2	('unclothed.a.01', 2603)	unappareled unattired unclad undressed ungarbed ungarmented
undue-a-1	('immoderate.a.01', 1136)	undue
undue-a-2	('immoderate.a.01', 2272)	undue
undue-a-3	('unreasonable.a.01', 3333)	undue unjustified unwarranted
undue-a-4	('immoderate.a.01', 5962)	excessive inordinate undue unreasonable
uneasy-a-1	('uneasy.a.01', 666)	uneasy
uneasy-a-2	('restless.a.03', 864)	restless uneasy
uneasy-a-3	('troubled.a.01', 14811)	anxious nervous queasy uneasy unquiet
uneasy-a-4	('uncomfortable.a.01', 3500)	awkward ill_at_ease uneasy
unemployed-a-1	('unemployed.a.01', 931)	unemployed
unemployed-n-1	('people.n.01', 1585)	unemployed_people unemployed
unemployment-n-1	('state.n.02', 2361)	unemploym

vivid-a-1	('realistic.a.01', 7665)	graphic lifelike pictorial vivid
vivid-a-2	('colorful.a.01', 608)	vivid
vivid-a-3	('colorful.a.01', 3353)	bright brilliant vivid
vivid-a-4	('colorful.a.01', 733)	intense vivid
vividly-r-1	('realistic.a.01', 424)	vividly
vocabulary-n-1	('book.n.01', 322)	vocabulary
vocabulary-n-2	('cognition.n.01', 1427)	vocabulary lexicon mental_lexicon
vocabulary-n-3	('system.n.04', 1360)	vocabulary
vocation-n-1	('activity.n.01', 1818)	career calling vocation
vocation-n-2	('body.n.02', 744)	occupational_group vocation
vocational-a-1	('vocational.a.01', 728)	vocational
voice-n-1	('person.n.01', 1365)	voice
voice-n-2	('communication.n.02', 2500)	voice vocalization vocalisation vocalism phonation vox
voice-n-3	('sound.n.04', 1458)	voice
voice-n-4	('act.n.02', 2457)	articulation voice
voice-n-5	('act.n.02', 996)	voice
voice-n-6	('property.n.02', 576)	voice
voice-n-7	('singer.n.01', 833)	voice
voice-n-8	('person.n.01', 9538)	spokesperson interpreter representative voice
v